<a href="https://colab.research.google.com/github/zaidalyafeai/Notebooks/blob/master/Strokes_QuickDraw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The Quick Draw Dataset is a collection of 50 million drawings across 345 categories, contributed by players of the game Quick Draw. 

![alt text](https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/preview.jpg)

# The  Raw Dataset

This table shows a description of the fields of each entry in the dataset

>Key | Type | Description
>--- | ---
>key_id 	| 64-bit unsigned integer  |	A unique identifier across all drawings.
> word |	string 	|Category the player was prompted to draw.
>recognized |	boolean 	|Whether the word was recognized by the game.
> timestamp 	| datetime 	| When the drawing was created.
> countrycode |	string |	A two letter country code 
> drawing |	string |	A JSON array representing the vector drawing

# Imports

In [0]:
import os
import io
import random
import glob
import math
import base64
import json
import numpy as np
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# Download the Dataset 

Loop over the classes and download the currospondent data. We only download 10 classes for visualization. 

In [0]:
!mkdir data
classes = ['table', 'sun', 'laptop', 'face', 'pants', 'ladder', 'eyeglasses', 'camera', 'sword', 'cat']

In [0]:
def download():  
  #base link  
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/'
  
  #download each class as json files 
  for c in classes:
    path = f'{base}raw/{c}.ndjson'
    print(path)
    urllib.request.urlretrieve(path, f'data/{c}.ndjson')

In [6]:
download() 

https://storage.googleapis.com/quickdraw_dataset/full/raw/table.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/sun.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/laptop.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/face.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/pants.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/ladder.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/eyeglasses.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/camera.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/sword.ndjson
https://storage.googleapis.com/quickdraw_dataset/full/raw/cat.ndjson


# Load to Memory 

Load the `drawing` information for each file. Each drawing contains a number of strokes and each stroke contain the array $[x, y, t]$ where $x,y$ are the coordinates as array and $t$ is the time stamps. 

In [0]:
drawings = []

files = os.listdir('data')

i = 0 

for file in files:
  contents = open(f'data/{file}', "r").read() 
  data = contents.split('\n')
  
  #load samples for each class 
  for h in data[:5]:
    drawings.append(json.loads(h)['drawing'])
  i += 1

In [0]:
#the first stroke of the first drawing
[x, y, t] = drawings[0][0]

# Animation

In [0]:
def create_animation(drawing, fps = 30, idx = 0, lw = 5): 
  
  seq_length = 0 
  
  xmax = 0 
  ymax = 0 
  
  xmin = math.inf
  ymin = math.inf
  
  #retreive min,max and the length of the drawing  
  for k in range(0, len(drawing)):
    x = drawing[k][0]
    y = drawing[k][1]

    seq_length += len(x)
    xmax = max([max(x), xmax]) 
    ymax = max([max(y), ymax]) 
    
    xmin = min([min(x), xmin]) 
    ymin = min([min(y), ymin]) 
    
  i = 0 
  j = 0
  
  # First set up the figure, the axis, and the plot element we want to animate
  fig = plt.figure()
  ax = plt.axes(xlim=(xmax+lw, xmin-lw), ylim=(ymax+lw, ymin-lw))
  ax.set_facecolor("white")
  line, = ax.plot([], [], lw=lw)

  #remove the axis 
  ax.grid = False
  ax.set_xticks([])
  ax.set_yticks([])
  
  # initialization function: plot the background of each frame
  def init():
      line.set_data([], [])
      return line, 

  # animation function.  This is called sequentially
  def animate(frame):    
    nonlocal i, j, line
    x = drawing[i][0]
    y = drawing[i][1]
    line.set_data(x[0:j], y[0:j])
    
    if j >= len(x):
      i +=1
      j = 0 
      line, = ax.plot([], [], lw=lw)
      
    else:
      j += 1
    return line,
  
  # call the animator.  blit=True means only re-draw the parts that have changed.
  anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames= seq_length + len(drawing), blit=True)
  plt.close()
  
  # save the animation as an mp4.  
  anim.save(f'video.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])

In [0]:
#create animation for a random drawing 
drawing = drawings[np.random.randint(0, len(drawings))]
create_animation(drawing)

In [46]:
video = io.open('video.mp4', 'r+b').read()

encoded = base64.b64encode(video)
HTML(data='''<video alt="video" autoplay loop>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))